W tym notebooku importujemy Keras oraz bibliotekę reinforcement learning i korzystamy z backendu Tensorflow. Do rozwiązania problemu przedstawionego w poprzednim notebooku użyjemy algorytmu Deep Q-learning, który dostosuje wagi na podstawie danych ze środowiska.

In [1]:
import gym
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

Using TensorFlow backend.


nb_actions będzie zawierać informacje o wszystkich dostępnych możliwych akcjach.

In [2]:
env = gym.make('CartPole-v0')
np.random.seed(1)
env.seed(1)
nb_actions = env.action_space.n

Tutaj budujemy faktyczny model sekwencyjny i dodajemy do niego kolejne warstwy, np. flatten zmienia macierz w prosty array. Po czym klasyczne warstwy Dense, z 20 neuronami i rektyfikowaną jednostką liniową jako naszą funkcje aktywacyjną do każdej z warstw. Gdzie na końcu liczba neuronów będzie równa zmiennej nb_actions określonej wcześniej, przy liniowej funkcji aktywacyjnej (rektyfikowana nie ma tu sensu).

In [3]:
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(20))
model.add(Activation('relu'))
model.add(Dense(20))
model.add(Activation('relu'))
model.add(Dense(20))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
model.summary()

W0207 01:35:35.586518 13380 deprecation_wrapper.py:119] From C:\Users\Patryk\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0207 01:35:35.605467 13380 deprecation_wrapper.py:119] From C:\Users\Patryk\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0207 01:35:35.617435 13380 deprecation_wrapper.py:119] From C:\Users\Patryk\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                100       
_________________________________________________________________
activation_1 (Activation)    (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                420       
_________________________________________________________________
activation_2 (Activation)    (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 20)                420       
_________________________________________________________________
activation_3 (Activation)    (None, 20)                0         
__________

Jak widać powyżej, nasza sieć neuronowa jest już gotowa do użycia. Teraz należy skonfigurować algorytm reinforcement learning. Zmienną memory tworzymy po to, żeby nasz agent DQN zapamiętywał poprzednie akcje w czasie.

In [4]:
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model = model,
               nb_actions = nb_actions,
               memory = memory,
               nb_steps_warmup = 10,
               target_model_update = 1e-2,
               policy = policy)

Następnie dokonujemy kompilacji przy użyciu algorytmu optymalizującego Adam, który jest obecnie najczęściej używany. Przy czym błąd określany jest przez mean absolute error.

In [5]:
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

W0207 01:35:35.724150 13380 deprecation_wrapper.py:119] From C:\Users\Patryk\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0207 01:35:35.724150 13380 deprecation_wrapper.py:119] From C:\Users\Patryk\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0207 01:35:37.597707 13380 deprecation_wrapper.py:119] From C:\Users\Patryk\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Tutaj trenujemy naszą sieć w środowisku CartPole.

In [6]:
dqn.fit(env, nb_steps=1000, visualize=True, verbose=2)

Training for 1000 steps ...


C:\Users\Patryk\Anaconda3\lib\site-packages\rl\memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


  19/1000: episode: 1, duration: 1.534s, episode steps: 19, steps per second: 12, episode reward: 19.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.368 [0.000, 1.000], mean observation: 0.066 [-1.176, 1.899], loss: 0.478723, mean_absolute_error: 0.578019, mean_q: 0.211937
  36/1000: episode: 2, duration: 0.141s, episode steps: 17, steps per second: 121, episode reward: 17.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.471 [0.000, 1.000], mean observation: -0.107 [-0.984, 0.363], loss: 0.326331, mean_absolute_error: 0.575049, mean_q: 0.458034


C:\Users\Patryk\Anaconda3\lib\site-packages\rl\memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


  45/1000: episode: 3, duration: 0.076s, episode steps: 9, steps per second: 118, episode reward: 9.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.222 [0.000, 1.000], mean observation: 0.148 [-1.182, 1.984], loss: 0.236224, mean_absolute_error: 0.593429, mean_q: 0.689771
  59/1000: episode: 4, duration: 0.115s, episode steps: 14, steps per second: 121, episode reward: 14.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.357 [0.000, 1.000], mean observation: 0.083 [-0.845, 1.586], loss: 0.179544, mean_absolute_error: 0.686464, mean_q: 1.029197
  79/1000: episode: 5, duration: 0.165s, episode steps: 20, steps per second: 121, episode reward: 20.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.350 [0.000, 1.000], mean observation: 0.094 [-1.185, 2.169], loss: 0.152595, mean_absolute_error: 0.715630, mean_q: 1.196697
  97/1000: episode: 6, duration: 0.151s, episode steps: 18, steps per second: 119, episode reward: 18.000, mean reward: 1.000 [1.000, 1.000], mean action: 0

 526/1000: episode: 33, duration: 0.124s, episode steps: 15, steps per second: 120, episode reward: 15.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.533 [0.000, 1.000], mean observation: -0.100 [-1.327, 0.784], loss: 0.202913, mean_absolute_error: 2.581398, mean_q: 5.019850
 538/1000: episode: 34, duration: 0.099s, episode steps: 12, steps per second: 121, episode reward: 12.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.250 [0.000, 1.000], mean observation: 0.100 [-1.388, 2.153], loss: 0.190828, mean_absolute_error: 2.677284, mean_q: 5.217380
 549/1000: episode: 35, duration: 0.092s, episode steps: 11, steps per second: 120, episode reward: 11.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.727 [0.000, 1.000], mean observation: -0.138 [-1.803, 0.963], loss: 0.187702, mean_absolute_error: 2.733887, mean_q: 5.365613
 570/1000: episode: 36, duration: 0.175s, episode steps: 21, steps per second: 120, episode reward: 21.000, mean reward: 1.000 [1.000, 1.000], mean a

Wagi które uzyskaliśmy w wyniku trenowania zapisuje w osobnym pliku.

In [7]:
dqn.save_weights('dqn_{}_weights.h5f'.format('CartPole-v0'), overwrite=True)

Dokonujemy testu sieci przy 10 iteracjach.

In [8]:
dqn.test(env, nb_episodes=10, visualize=True)

Testing for 10 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 173.000, steps: 173
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 184.000, steps: 184
Episode 5: reward: 161.000, steps: 161
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 157.000, steps: 157
Episode 8: reward: 182.000, steps: 182
Episode 9: reward: 162.000, steps: 162
Episode 10: reward: 200.000, steps: 200


Algorytm DQN dopasował idealnie wszystkie wagi do funkcji i uzyskujemy maksymalny wynik możliwy dla ilości momentów w czasie.
Możemy również zauważyć, że korzystanie z możliwosći karty graficznej do trenowania tej sieci jest nieporównywalnie szybsze od poprzedniej metody w notebooku nr 1.